import os
os.environ['PATH']
stream = os.popen('owltools --help')
output = stream.read()
output

In [214]:
import pandas as pd
import os
import glob
from rdflib import Graph
from urllib.request import urlopen
import subprocess
import time
from IPython.display import Markdown as md
from IPython.display import HTML
from pathlib import Path


# use ontobio!!
# pd.set_option('display.notebook_repr_html',False)
# https://github.com/jupyter/nbconvert/issues/946

In [215]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Mondo analysis

from ontobio.ontol_factory import OntologyFactory
mondo_file="/Users/matentzn/ws/mondo/src/ontology/mondo-qc.obo"
ofa = OntologyFactory()
ont = ofa.create(mondo_file)
ont.get_roots()[:3]

In [216]:
warnings = [Path(warn).stem for warn in glob.glob('../../sparql/*-warning.sparql')] 
NUMBER_ERRORS_SHOWN=15

mondo="http://purl.obolibrary.org/obo/mondo.owl"
g = Graph()
g.parse("../mondo-qc.owl", format="xml")

In [217]:
def get_report(name):
    report_files= glob.glob('*-%s.tsv' %(name))
    reports = []
    for f in report_files:
        try:
            df = pd.read_csv(f,sep="\t")
            df['ontology']=Path(f).stem.replace("-"+name,"")
            df.columns = [str(col).replace("?","") for col in df.columns]
            df.replace('[<]http[:][/][/]purl[.]obolibrary[.]org[/]obo[/]MONDO[_]','MONDO:', inplace=True,regex=True)
            df.replace('[>]$','', inplace=True, regex=True)
            reports.append(df)
        except pd.errors.EmptyDataError:
            print("WARNING! ", name, " is empty and has been skipped.")
    if reports:
        return pd.concat(reports)
    return pd.DataFrame(columns=['ontology'])

## Robot report analysis

In [218]:
df = get_report("robot-report-obo")
# Bug in ROBOT, wrong number of columns:
df.columns=['Rule Name', 'Subject', 'Property', 'Value','Trash','Ontology']
df['Level'] = df.index
df.reset_index(level=0, inplace=True)
del df['index']
del df['Trash']
df

,Rule Name,Subject,Property,Value,Ontology,Level
0,multiple_equivalent_classes,MONDO:0021108,owl:equivalentClass,ee3c0de2-7e9a-42ee-b351-bfc17e83205dgenid312398,mondo-edit.obo,ERROR
1,multiple_equivalent_classes,MONDO:0021108,owl:equivalentClass,ee3c0de2-7e9a-42ee-b351-bfc17e83205dgenid312402,mondo-edit.obo,ERROR
2,multiple_equivalent_classes,MONDO:0045024,owl:equivalentClass,ee3c0de2-7e9a-42ee-b351-bfc17e83205dgenid339146,mondo-edit.obo,ERROR
3,multiple_equivalent_classes,MONDO:0045024,owl:equivalentClass,ee3c0de2-7e9a-42ee-b351-bfc17e83205dgenid339150,mondo-edit.obo,ERROR
4,missing_ontology_description,mondo.owl,dc11:description,NaN,mondo-edit.obo,ERROR
...,...,...,...,...,...,...
271725,lowercase_definition,MONDO:0025085,IAO:0000115,inflammation of the liver in animals due to vi...,mondo.owl,INFO
271726,lowercase_definition,MONDO:0025100,IAO:0000115,inflammation of the udder in cows.,mondo.owl,INFO
271727,lowercase_definition,MONDO:0044255,IAO:0000115,<Subhead> Genetic Heterogeneity of Variation i...,mondo.owl,INFO
271728,lowercase_definition,MONDO:0044266,IAO:0000115,"{2,3:Berg and Bearn (1966, 1966)} discovered a...",mondo.owl,INFO


## Overview of errors

This breaks down the errors in the generic ROBOT report. _Note that the whole report is filtered to only show results pertaining to MONDO classes themselves_. All tables are cut off at {{ NUMBER_ERRORS_SHOWN }} rows.

In [219]:
df=df[df['Subject'].str.startswith('MONDO')]
df[['Level','Rule Name','Value','Ontology']].groupby(['Ontology','Level','Rule Name']).count()

Value
Ontology                  Level Rule Name                         
mondo-edit.obo            ERROR multiple_equivalent_classes      4
                          INFO  lowercase_definition           300
                                missing_superclass               0
                          WARN  duplicate_exact_synonym       1655
                                duplicate_scoped_synonym      1595
                                missing_definition               0
mondo.owl                 ERROR deprecated_class_reference      54
                                multiple_equivalent_classes  37563
                          INFO  lowercase_definition           300
                          WARN  duplicate_exact_synonym       1655
                                duplicate_scoped_synonym      1595
                                equivalent_pair              37603
                                missing_definition               0
mondo_current_release.owl ERROR deprecated_class_reference      60
                                duplicate_definition             2
                                multiple_equivalent_classes  37565
                          INFO  lowercase_definition           300
                          WARN  duplicate_exact_synonym       1659
                                duplicate_scoped_synonym      1597
                                equivalent_pair              37604
                                missing_definition               0

In [220]:
for o in df['Ontology'].unique():
    display(md("### %s" % o))
    dfo = df[df['Ontology']==o]
    for rule in dfo['Rule Name'].unique():
        display(md("#### %s" % rule))
        dfd = dfo[dfo['Rule Name']==rule].head(NUMBER_ERRORS_SHOWN)
        display(HTML(dfd.to_html(index=False)))


# Revealed a more complex issue we need to sort first: https://github.com/monarch-initiative/mondo/issues/1979
# https://github.com/ontodev/robot/issues/733 # Ignore 0021108 0045024 until ticket resolved.

### mondo-edit.obo

#### multiple_equivalent_classes

Rule Name,Subject,Property,Value,Ontology,Level
multiple_equivalent_classes,MONDO:0021108,owl:equivalentClass,ee3c0de2-7e9a-42ee-b351-bfc17e83205dgenid312398,mondo-edit.obo,ERROR
multiple_equivalent_classes,MONDO:0021108,owl:equivalentClass,ee3c0de2-7e9a-42ee-b351-bfc17e83205dgenid312402,mondo-edit.obo,ERROR
multiple_equivalent_classes,MONDO:0045024,owl:equivalentClass,ee3c0de2-7e9a-42ee-b351-bfc17e83205dgenid339146,mondo-edit.obo,ERROR
multiple_equivalent_classes,MONDO:0045024,owl:equivalentClass,ee3c0de2-7e9a-42ee-b351-bfc17e83205dgenid339150,mondo-edit.obo,ERROR


#### duplicate_exact_synonym

Rule Name,Subject,Property,Value,Ontology,Level
duplicate_exact_synonym,MONDO:0003401,oboInOwl:hasExactSynonym,yolk Sac tumor of the CNS,mondo-edit.obo,WARN
duplicate_exact_synonym,MONDO:0016739,oboInOwl:hasExactSynonym,yolk Sac tumor of the CNS,mondo-edit.obo,WARN
duplicate_exact_synonym,MONDO:0003404,oboInOwl:hasExactSynonym,yolk Sac tumor,mondo-edit.obo,WARN
duplicate_exact_synonym,MONDO:0005744,oboInOwl:hasExactSynonym,yolk Sac tumor,mondo-edit.obo,WARN
duplicate_exact_synonym,MONDO:0010420,oboInOwl:hasExactSynonym,XLP,mondo-edit.obo,WARN
duplicate_exact_synonym,MONDO:0010627,oboInOwl:hasExactSynonym,XLP,mondo-edit.obo,WARN
duplicate_exact_synonym,MONDO:0017304,oboInOwl:hasExactSynonym,XLOA,mondo-edit.obo,WARN
duplicate_exact_synonym,MONDO:0021019,oboInOwl:hasExactSynonym,XLOA,mondo-edit.obo,WARN
duplicate_exact_synonym,MONDO:0010209,oboInOwl:hasExactSynonym,xanthine oxidase deficiency,mondo-edit.obo,WARN
duplicate_exact_synonym,MONDO:0018106,oboInOwl:hasExactSynonym,xanthine oxidase deficiency,mondo-edit.obo,WARN


...

Rule Name,Subject,Property,Value,Ontology,Level
missing_definition,MONDO:0000005,IAO:0000115,NaN,mondo.owl,WARN
missing_definition,MONDO:0000009,IAO:0000115,NaN,mondo.owl,WARN
missing_definition,MONDO:0000014,IAO:0000115,NaN,mondo.owl,WARN
missing_definition,MONDO:0000023,IAO:0000115,NaN,mondo.owl,WARN
missing_definition,MONDO:0000030,IAO:0000115,NaN,mondo.owl,WARN
missing_definition,MONDO:0000032,IAO:0000115,NaN,mondo.owl,WARN
missing_definition,MONDO:0000045,IAO:0000115,NaN,mondo.owl,WARN
missing_definition,MONDO:0000049,IAO:0000115,NaN,mondo.owl,WARN
missing_definition,MONDO:0000050,IAO:0000115,NaN,mondo.owl,WARN
missing_definition,MONDO:0000060,IAO:0000115,NaN,mondo.owl,WARN


#### duplicate_scoped_synonym

Rule Name,Subject,Property,Value,Ontology,Level
duplicate_scoped_synonym,MONDO:0000155,oboInOwl:hasExactSynonym,rare inborn error of sequestering of triglyceride,mondo.owl,WARN
duplicate_scoped_synonym,MONDO:0000155,oboInOwl:hasRelatedSynonym,rare inborn error of sequestering of triglyceride,mondo.owl,WARN
duplicate_scoped_synonym,MONDO:0000242,oboInOwl:hasExactSynonym,dermatophytosis of beard,mondo.owl,WARN
duplicate_scoped_synonym,MONDO:0000242,oboInOwl:hasRelatedSynonym,dermatophytosis of beard,mondo.owl,WARN
duplicate_scoped_synonym,MONDO:0000270,oboInOwl:hasExactSynonym,disorder of lower respiratory tract,mondo.owl,WARN
duplicate_scoped_synonym,MONDO:0000270,oboInOwl:hasRelatedSynonym,disorder of lower respiratory tract,mondo.owl,WARN
duplicate_scoped_synonym,MONDO:0000330,oboInOwl:hasExactSynonym,endemic typhus Fever,mondo.owl,WARN
duplicate_scoped_synonym,MONDO:0000330,oboInOwl:hasRelatedSynonym,endemic typhus Fever,mondo.owl,WARN
duplicate_scoped_synonym,MONDO:0000351,oboInOwl:hasExactSynonym,rare inborn error of methionine catabolic process,mondo.owl,WARN
duplicate_scoped_synonym,MONDO:0000351,oboInOwl:hasRelatedSynonym,rare inborn error of methionine catabolic process,mondo.owl,WARN


#### lowercase_definition

Rule Name,Subject,Property,Value,Ontology,Level
lowercase_definition,MONDO:0000712,IAO:0000115,frontotemporal dementia plus amyotrophic lateral sclerosis.,mondo.owl,INFO
lowercase_definition,MONDO:0001554,IAO:0000115,secondary glaucoma caused by either excessive size or spheric shape of the lens.,mondo.owl,INFO
lowercase_definition,MONDO:0001709,IAO:0000115,sarcoidosis with a complication of hypercalcemia.,mondo.owl,INFO
lowercase_definition,MONDO:0001875,IAO:0000115,inflammation of the lateral epicondyle.,mondo.owl,INFO
lowercase_definition,MONDO:0002121,IAO:0000115,neuritis of a single nerve.,mondo.owl,INFO
lowercase_definition,MONDO:0004135,IAO:0000115,thyroiditis associated with painless enlargement of the thyroid gland. It occurs more frequently in females and is characterized by alterations between hyperthyroidism and hypothyroidism and the eventual return to normal thyroid gland function.,mondo.owl,INFO
lowercase_definition,MONDO:0005196,IAO:0000115,presence of structurally anomalous spermatozoa in the semen; malformations include the physical bending of the sperm to produce kinks or bends,mondo.owl,INFO
lowercase_definition,MONDO:0005448,IAO:0000115,liver injury resulting from hepatitis C infection,mondo.owl,INFO
lowercase_definition,MONDO:0005483,IAO:0000115,hair loss as a result of chemotherapy treatment,mondo.owl,INFO
lowercase_definition,MONDO:0005490,IAO:0000115,stroke caused by the blockage of blood flow in one of the large arteries feeding the brain,mondo.owl,INFO


## Other Quality Control checks

In [226]:
summary_data = []

for r in warnings:
    df_r = get_report(r)
    cols = df_r.columns.tolist()
    cols_term = df_r.columns.tolist()
    if 'term' in cols:
        cols.remove('term')
    if 'term_label' in cols:
        cols.remove('term_label')
        cols_term.remove('term_label')
    df_summary=df_r[cols_term].groupby(cols).count()
    df_summary=df_summary.add_suffix('_Count').reset_index()
    display(HTML(df_summary.head(3).to_html(index=False)))
    display(md("### Check: %s" % r))
    for o in df_r['ontology'].unique():
        df_ro=df_r[df_r['ontology']==o]
        display(md("#### %s (Errors: %d)" % (o,len(df_ro))))
        summary_data.append([r,o,len(df_ro)])
        if (len(df_ro)>0):
            df_rd = df_ro.head(NUMBER_ERRORS_SHOWN).copy()
            display(HTML(df_rd.to_html(index=False)))

df_summary_data = pd.DataFrame(summary_data,columns=['check','ontology','errors'])
display(HTML(df_summary_data.to_html(index=False)))

d2,d2_label,ontology,term_Count
MONDO:0008648,"ventricular tachycardia, familial",mondo.owl,4
MONDO:0008648,"ventricular tachycardia, familial",mondo_current_release.owl,4
MONDO:0009861,phenylketonuria,mondo-edit.obo,1


### Check: no-subclass-between-genetic-disease-warning

#### mondo_current_release.owl (Errors: 68)

term,term_label,d2,d2_label,ontology
MONDO:0013529,catecholaminergic polymorphic ventricular tachycardia 3,MONDO:0008648,"ventricular tachycardia, familial",mondo_current_release.owl
MONDO:0013966,catecholaminergic polymorphic ventricular tachycardia 4,MONDO:0008648,"ventricular tachycardia, familial",mondo_current_release.owl
MONDO:0012160,spondylometaphyseal dysplasia-cone-rod dystrophy syndrome,MONDO:0009943,Pyle disease,mondo_current_release.owl
MONDO:0009709,"myopathy, centronuclear, 2",MONDO:0100175,TTN-related myopathy,mondo_current_release.owl
MONDO:0014191,catecholaminergic polymorphic ventricular tachycardia 5,MONDO:0008648,"ventricular tachycardia, familial",mondo_current_release.owl
MONDO:0007794,hypogonadotropic hypogonadism 7 with or without anosmia,MONDO:0014107,hypogonadotropic hypogonadism 21 with or without anosmia,mondo_current_release.owl
MONDO:0007794,hypogonadotropic hypogonadism 7 with or without anosmia,MONDO:0014105,hypogonadotropic hypogonadism 19 with or without anosmia,mondo_current_release.owl
MONDO:0007794,hypogonadotropic hypogonadism 7 with or without anosmia,MONDO:0013926,hypogonadotropic hypogonadism 14 with or without anosmia,mondo_current_release.owl
MONDO:0007794,hypogonadotropic hypogonadism 7 with or without anosmia,MONDO:0014106,hypogonadotropic hypogonadism 20 with or without anosmia,mondo_current_release.owl
MONDO:0007794,hypogonadotropic hypogonadism 7 with or without anosmia,MONDO:0014103,hypogonadotropic hypogonadism 18 with or without anosmia,mondo_current_release.owl


#### mondo-edit.obo (Errors: 13)

...

#### mondo-edit.obo (Errors: 200)

term,term_label,related,ontology
MONDO:0021010,skin lymphangiosarcoma,lymphangiosarcoma of Stewart and Treves,mondo-edit.obo
MONDO:0000270,lower respiratory tract disease,disorder of lower respiratory tract,mondo-edit.obo
MONDO:0008738,"aganglionosis, total intestinal",total intestinal aganglionosis,mondo-edit.obo
MONDO:0021534,rectal neuroendocrine tumor G1,rectum carcinoid tumor,mondo-edit.obo
MONDO:0015564,Castleman disease,angiofollicular lymph node hyperplasia,mondo-edit.obo
MONDO:0005289,paranasal sinus neoplasm (disease),tumor of paranasal sinus,mondo-edit.obo
MONDO:0006266,Leydig cell tumor,interstitial cell neoplasm,mondo-edit.obo
MONDO:0024467,apocrine sweat gland disease,disorder of apocrine sweat gland,mondo-edit.obo
MONDO:0002540,childhood oligodendroglioma,pediatric oligodendroglioma,mondo-edit.obo
MONDO:0002485,breast neuroendocrine neoplasm,breast neuroendocrine tumor,mondo-edit.obo


#### mondo.owl (Errors: 200)

term,term_label,related,ontology
MONDO:0021010,skin lymphangiosarcoma,lymphangiosarcoma of Stewart and Treves,mondo.owl
MONDO:0000270,lower respiratory tract disease,disorder of lower respiratory tract,mondo.owl
MONDO:0008738,"aganglionosis, total intestinal",total intestinal aganglionosis,mondo.owl
MONDO:0021534,rectal neuroendocrine tumor G1,rectum carcinoid tumor,mondo.owl
MONDO:0015564,Castleman disease,angiofollicular lymph node hyperplasia,mondo.owl
MONDO:0005289,paranasal sinus neoplasm (disease),tumor of paranasal sinus,mondo.owl
MONDO:0006266,Leydig cell tumor,interstitial cell neoplasm,mondo.owl
MONDO:0024467,apocrine sweat gland disease,disorder of apocrine sweat gland,mondo.owl
MONDO:0002540,childhood oligodendroglioma,pediatric oligodendroglioma,mondo.owl
MONDO:0002485,breast neuroendocrine neoplasm,breast neuroendocrine tumor,mondo.owl


check,ontology,errors
no-subclass-between-genetic-disease-warning,mondo_current_release.owl,68
no-subclass-between-genetic-disease-warning,mondo-edit.obo,13
no-subclass-between-genetic-disease-warning,mondo.owl,68
single-child-warning,mondo.owl,1558
single-child-warning,mondo_current_release.owl,1559
single-child-warning,mondo-edit.obo,1385
related-exact-synonym-warning,mondo_current_release.owl,200
related-exact-synonym-warning,mondo-edit.obo,200
related-exact-synonym-warning,mondo.owl,200


## Mondo Statistics

In [222]:
df = get_report("term-stats")
df.drop_duplicates(inplace=True)

## Relationships

In [223]:
props=df[['term','property','ontology']].groupby(['ontology','property']).count().add_suffix('_count').reset_index()
props.sort_values(inplace=True, by=['ontology','term_count'], ascending=False)
props_pivot=props.pivot(index='property', columns='ontology', values='term_count')
props_pivot = pd.DataFrame(props_pivot.to_records())
props_pivot.sort_values(by=['mondo.owl'],ascending=False,inplace=True)
display(md("Table: Breakdown of the number of relationships in Mondo."))
display(HTML(props_pivot.to_html(index=False)))

Table: Breakdown of the number of relationships in Mondo.

property,mondo-edit.obo,mondo.owl,mondo_current_release.owl
<http://www.geneontology.org/formats/oboInOwl#hasDbXref,128445.0,99967.0,99965.0
<http://www.w3.org/2004/02/skos/core#exactMatch,NaN,70551.0,70550.0
<http://www.geneontology.org/formats/oboInOwl#hasExactSynonym,64369.0,64369.0,64352.0
<http://www.w3.org/2000/01/rdf-schema#subClassOf,46999.0,47546.0,47538.0
<http://www.w3.org/2002/07/owl#equivalentClass,7403.0,45006.0,45007.0
<http://www.geneontology.org/formats/oboInOwl#hasRelatedSynonym,37874.0,37874.0,37872.0
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type,24231.0,24231.0,24229.0
<http://www.w3.org/2000/01/rdf-schema#label,24114.0,24114.0,24112.0
<http://www.geneontology.org/formats/oboInOwl#id,24114.0,24114.0,24112.0
<http://www.w3.org/2004/02/skos/core#closeMatch,NaN,16509.0,16509.0


In [224]:
current_to_release_diff=props_pivot[['property', 'mondo.owl', 'mondo_current_release.owl']]
current_to_release_diff=current_to_release_diff[current_to_release_diff['mondo.owl']!=current_to_release_diff['mondo_current_release.owl']]
current_to_release_diff.dropna(inplace=True)
current_to_release_diff
current_to_release_diff['difference']=current_to_release_diff['mondo.owl']-current_to_release_diff['mondo_current_release.owl']
current_to_release_diff.sort_values(by=['difference'],ascending=False,inplace=True)
current_to_release_diff[['mondo.owl', 'mondo_current_release.owl','difference']] = current_to_release_diff[['mondo.owl', 'mondo_current_release.owl','difference']].astype(int)
display(HTML(current_to_release_diff.to_html(index=False)))

property,mondo.owl,mondo_current_release.owl,difference
<http://www.geneontology.org/formats/oboInOwl#hasExactSynonym,64369,64352,17
<http://www.w3.org/2000/01/rdf-schema#subClassOf,47546,47538,8
<http://www.geneontology.org/formats/oboInOwl#hasBroadSynonym,486,483,3
<http://www.geneontology.org/formats/oboInOwl#hasDbXref,99967,99965,2
<http://www.geneontology.org/formats/oboInOwl#hasRelatedSynonym,37874,37872,2
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type,24231,24229,2
<http://www.w3.org/2000/01/rdf-schema#label,24114,24112,2
<http://www.geneontology.org/formats/oboInOwl#id,24114,24112,2
<http://www.geneontology.org/formats/oboInOwl#created_by,65,63,2
<http://www.w3.org/2004/02/skos/core#exactMatch,70551,70550,1
